# PACO Time-Based Simulation Notebook

This notebook demonstrates **TimeStrategy** which advances by a fixed time step (1 unit per step).

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("src"))
sys.path.append(os.path.abspath("simulator/src"))

from paco.parser.bpmn_parser import create_parse_tree
from src.utils.env import EXPRESSION, H, IMPACTS, DURATIONS, IMPACTS_NAMES, PROBABILITIES, DELAYS, LOOP_PROBABILITY, LOOP_ROUND

from model.region import RegionModel
from model.context import NetContext
from model.status import ActivityState
from strategy.time import TimeStrategy  # Using TimeStrategy instead of CounterExecution

from paco.parser.dot.bpmn import get_bpmn_dot_from_parse_tree

import graphviz
from IPython.display import display, SVG
from pm4py.visualization.petri_net import visualizer as pn_visualizer

print("Libraries imported.")

In [ ]:
bpmn_def = {
    EXPRESSION: "Cutting, ((Bending, (Heavy_Polishing ^[N1] Light_Polishing)) || (Milling, (Fine_Deposition / [C1] Rough_Deposition))), (HPHS / [C2] LPLS)",
    H: 0,
    IMPACTS: {
        "Cutting": [10, 2], "Bending": [5, 1], 
        "Heavy_Polishing": [8, 3], "Light_Polishing": [4, 1],
        "Milling": [6, 2],
        "Fine_Deposition": [7, 2], "Rough_Deposition": [5, 1],
        "HPHS": [3, 1], "LPLS": [2, 1]
    },
    DURATIONS: {
        "Cutting": 2.0, "Bending": 1.5, 
        "Heavy_Polishing": 2.0, "Light_Polishing": 1.0,
        "Milling": 3.0,
        "Fine_Deposition": 2.5, "Rough_Deposition": 1.5,
        "HPHS": 1.0, "LPLS": 0.5
    },
    IMPACTS_NAMES: ["Energy", "Labor"],
    PROBABILITIES: {"N1": 0.5},
    DELAYS: {"C1": 0, "C2": 0},
    LOOP_PROBABILITY: {}, 
    LOOP_ROUND: {}
}
print("BPMN Definition created with varied durations.")

In [ ]:
# Create Parse Tree and Simulator Context
parse_tree, _, _, _ = create_parse_tree(bpmn_def)
tree_dict = parse_tree.to_dict()
root_region = RegionModel(**tree_dict)

# Use TimeStrategy instead of CounterExecution
stm = TimeStrategy()
ctx = NetContext.from_region(root_region, stm)

# Build regions map
regions_map = {}
def build_region_dict(r):
    regions_map[int(r.id)] = r
    if r.children:
        for child in r.children:
            build_region_dict(child)
build_region_dict(root_region)

status = {r: ActivityState.WAITING for r in regions_map.values()}
marking = ctx.initial_marking
total_time = 0.0

print(f"Petri Net: {len(ctx.net.places)} places, {len(ctx.net.transitions)} transitions")
print("Using TimeStrategy (advances by time_step per call)")

In [ ]:
def show_bpmn(parse_tree_obj, status_dict, title="BPMN"):
    status_by_id = {str(r.id): s for r, s in status_dict.items()}
    dot_str = get_bpmn_dot_from_parse_tree(parse_tree_obj, bpmn_def[IMPACTS_NAMES], status_by_id)
    graph = graphviz.Source(dot_str)
    print(f"\n--- {title} ---")
    display(SVG(graph.pipe(format='svg')))

def show_petri_net(petri_net, current_marking, final_marking, title="Petri Net"):
    pm4py_marking = {}
    for place in current_marking.keys():
        token, _, _ = current_marking[place]
        pm4py_marking[place] = token
    gviz = pn_visualizer.apply(petri_net, pm4py_marking, final_marking)
    print(f"\n--- {title} ---")
    display(SVG(gviz.pipe(format='svg')))

def show_step(step_name, parse_tree_obj, status_dict, petri_net, current_marking, final_marking, curr_time):
    print(f"\n{'='*60}\n{step_name} | Total Time: {curr_time}\n{'='*60}")
    active = [r.label if r.label else f"ID:{r.id}" for r, s in status_dict.items() if s == ActivityState.ACTIVE]
    completed = [r.label for r, s in status_dict.items() if s == ActivityState.COMPLETED and r.is_task()]
    print(f"Active: {', '.join(active) if active else 'None'}")
    print(f"Completed: {', '.join(completed) if completed else 'None'}")
    show_bpmn(parse_tree_obj, status_dict, f"BPMN - {step_name}")
    show_petri_net(petri_net, current_marking, final_marking, f"Petri Net - {step_name}")

## Initial State

In [ ]:
show_step("Initial State", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)

## Step 1: Advance +1 time unit

In [ ]:
TIME_STEP = 1.0
marking, _, _, exec_time, _ = stm.consume(ctx, marking, regions_map, status, time_step=TIME_STEP)
total_time += exec_time
show_step("Step 1", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)

## Step 2: Advance +1 time unit

In [ ]:
marking, _, _, exec_time, _ = stm.consume(ctx, marking, regions_map, status, time_step=TIME_STEP, decisions=[])
total_time += exec_time
show_step("Step 2", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)

## Step 3: Advance +1 time unit

In [ ]:
marking, _, _, exec_time, _ = stm.consume(ctx, marking, regions_map, status, time_step=TIME_STEP, decisions=[])
total_time += exec_time
show_step("Step 3", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)

## Step 4: Advance +1 time unit

In [ ]:
marking, _, _, exec_time, _ = stm.consume(ctx, marking, regions_map, status, time_step=TIME_STEP, decisions=[])
total_time += exec_time
show_step("Step 4", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)

## Step 5: Advance +1 time unit

In [ ]:
marking, _, _, exec_time, _ = stm.consume(ctx, marking, regions_map, status, time_step=TIME_STEP, decisions=[])
total_time += exec_time
show_step("Step 5", parse_tree, status, ctx.net, marking, ctx.final_marking, total_time)